In [31]:
import re
import os
import pandas as pd
from fuzzywuzzy import fuzz
from math import sin, cos, sqrt, atan2, radians

In [39]:
d=hmis['district'].unique()

df = pd.read_csv('./Data/google_w_st_dist_9545.csv')

dd=[]
for i in df.iterrows():
    
    a1=i[1][-2]
    for j in d:
        a2=j.lower()
        if a1.lower()==a2:
            dd.append((i[0],j))
            break
    
            

S=pd.Series([x[1] for x in dd],index=[x[0] for x in dd])

df1=df.loc[S.index]

df1['district']=S

df2=df1.reset_index(drop=True)

df2.to_csv('./Data/google_w_st_dist_8283.csv',index=False)

df1 = pd.read_csv('./Data/hospitals_prod.csv')
df1 = hos_prod.loc[hos_prod['source'] == 'HMIS']

df2 = pd.read_csv('./Data/google_w_st_dist_8283.csv')

dis = list(df1['district'].unique())

for i in dis:
    d1=df1[df1['district']==i]
    d2=df2[df2['district']==i]
    if d2.shape[0]==0:
        continue
    else:
        print(d1['name_add'])
        print(d2['name_add'])
        break

## Cross  Product

In [26]:
def unique(sequence):
    
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def preprocess(text):
    
    text = re.sub('[^a-zA-Z0-9 ]','',text)
    text = ' '.join(unique(text.upper().split()))
    return text.strip()

def distance_calculator(long1, lat1 ,long2, lat2):
    
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(long1)
    lat2 = radians(lat2)
    lon2 = radians(long2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

def scores(string1,string2):
    
    ratio = fuzz.ratio(string1,string2)
    tokenset_ratio = fuzz.token_set_ratio(string1,string2)    
    tokensort_ratio = fuzz.token_sort_ratio(string1,string2)    
    partial_ratio = fuzz.partial_ratio(string1,string2)  
    total = sum([ratio,tokenset_ratio,tokensort_ratio,partial_ratio])
    return ratio,tokenset_ratio,tokensort_ratio,partial_ratio,total

### Data Pre Processing

In [14]:
hos_prod = pd.read_csv('./Data/hospitals_prod.csv')
hmis = hos_prod.loc[hos_prod['source'] == 'HMIS']
hmis['state'] = hmis['state'].str.lower().str.strip()
hmis['district'] = hmis['district'].str.lower().str.strip()
hmis['name'].fillna('-',inplace = True)
hmis['address'].fillna('-',inplace = True)
hmis['name'] = hmis['name'].str.lower().str.strip()
hmis['address'] = hmis['address'].str.lower().str.strip()
hmis['name_add_hmis'] = hmis[['name','address']].apply(lambda x: preprocess(' '.join(x.dropna().astype(str))),axis=1)
hmis = hmis.rename(columns = {'name':'name_hmis','address':'address_hmis','hospital_id':'hmis_id','lat':'lat_hmis',
                              'lng':'lng_hmis'})
hmis.replace({'state':{'dadra & nagar haveli':'the dadra and nagar haveli and daman and diu',
                       'daman & diu':'the dadra and nagar haveli and daman and diu' }},inplace = True)

hmis_ = hmis[['name_hmis','address_hmis','name_add_hmis','hmis_id','district','state','lat_hmis','lng_hmis']]

/home/Documents/karuna/Transliterate/Data/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (22,36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/Documents/karuna/Transliterate/Data/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/Documents/karuna/Transliterate/Data/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [24]:
google_st_dist = pd.read_csv('./Data/google_w_st_dist_8283.csv')
google_hmis = pd.read_csv('./Data/google_in_hmis_format.csv')
google = google_st_dist.merge(google_hmis[['hospital_id','lat','lng']], on = 'hospital_id')
google['name'].fillna('-',inplace = True)
google['address'].fillna('-',inplace = True)
google['state'] = google['state'].str.lower().str.strip()
google['district'] = google['district'].str.lower().str.strip()
google['name'] = google['name'].str.lower().str.strip()
google['address'] = google['address'].str.lower().str.strip()

google = google.rename(columns = {'name_add':'name_add_google','name':'name_google','address':'address_google',
                                  'hospital_id':'google_id','lat':'lat_google','lng':'lng_google'})
google.replace({'state':{'jammu and kashmir':'jammu & kashmir'}},inplace = True)

google_ = google[['name_google','address_google','name_add_google','google_id','district','state','lat_google','lng_google']]

### Scores and Distance Calculation on Siatrict filtered rows

In [34]:
google_s_grp = google_.groupby('state')
hmis_s_grp = hmis_.groupby('state')
state_list = ['himachal pradesh']

for s in state_list:
    
    if os.path.isdir('./Data/States/'+s) is False:
        os.mkdir('./Data/States/'+s)
    
    st_google = google_s_grp.get_group(s)
    st_hmis = hmis_s_grp.get_group(s)
    
    google_d_grp = st_google.groupby('district')
    hmis_d_grp = st_hmis.groupby('district')
    
    district_list = set(list(st_google['district'].unique())).intersection(set(list(st_hmis['district'].unique())))
    
    for d in district_list:
        
        print('State: ',s,'\tDistrict: ',d)
        if os.path.exists('./'+s+'/'+d+'.csv'):
            continue
        d_google = google_d_grp.get_group(d)
        d_hmis = hmis_d_grp.get_group(d)
            
        d_google['key'] = 0
        d_hmis['key'] = 0

        dfcross = d_google.merge(d_hmis,on = 'key',sort=True)
        print('Google: ',d_google.shape,'\tHMIS: ',d_hmis.shape,'\tCrossproduct: ',dfcross.shape)
        
        dfcross[['ratio','tokenset_ratio','tokensort_ratio','partial_ratio','total_score']] = dfcross.apply(lambda s:
                                                    pd.Series(scores(s['name_add_hmis'],s['name_add_google'])),axis = 1)
        dfcross[['ratio','tokenset_ratio','tokensort_ratio','partial_ratio','total_score']] = dfcross.apply(lambda s:
                                                    pd.Series(scores(s['name_hmis'],s['name_google'])),axis = 1)
        dfcross[['ratio','tokenset_ratio','tokensort_ratio','partial_ratio','total_score']] = dfcross.apply(lambda s:
                                                    pd.Series(scores(s['address_hmis'],s['address_google'])),axis = 1)
        dfcross['distance'] = dfcross.apply(lambda r: distance_calculator(r['lng_google'],r['lat_google'],
                                                                    r['lng_hmis'],r['lat_hmis']),axis = 1)

        dfcross.to_csv('./Data/States/'+s+'/'+d+'.csv',index = False)

State:  himachal pradesh 	District:  solan
Google:  (7, 9) 	HMIS:  (224, 9) 	Crossproduct:  (1568, 17)


/home/Documents/karuna/Transliterate/Data/venv/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/Documents/karuna/Transliterate/Data/venv/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


State:  himachal pradesh 	District:  mandi
Google:  (232, 9) 	HMIS:  (392, 9) 	Crossproduct:  (90944, 17)
State:  himachal pradesh 	District:  bilaspur
Google:  (29, 9) 	HMIS:  (160, 9) 	Crossproduct:  (4640, 17)
State:  himachal pradesh 	District:  kullu
Google:  (2, 9) 	HMIS:  (122, 9) 	Crossproduct:  (244, 17)
State:  himachal pradesh 	District:  una
Google:  (363, 9) 	HMIS:  (156, 9) 	Crossproduct:  (56628, 17)
State:  himachal pradesh 	District:  shimla
Google:  (13, 9) 	HMIS:  (338, 9) 	Crossproduct:  (4394, 17)
State:  himachal pradesh 	District:  chamba
Google:  (7, 9) 	HMIS:  (227, 9) 	Crossproduct:  (1589, 17)
State:  himachal pradesh 	District:  kangra
Google:  (6, 9) 	HMIS:  (528, 9) 	Crossproduct:  (3168, 17)
State:  himachal pradesh 	District:  hamirpur
Google:  (11, 9) 	HMIS:  (182, 9) 	Crossproduct:  (2002, 17)
